In [16]:
import os
import openai
import sys
sys.path.append("../..")

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())


In [17]:
from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
persist_directory = "documents/chroma"

In [18]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [19]:
print(vectordb._collection.count())

74


In [20]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [21]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [22]:
question = "Tell me about all-white mushrooms with large fruiting bodies."

In [23]:
smalldb.similarity_search(question, k=2)

[Document(id='adcaf1ee-3f76-4a78-b9ff-675e5a3514a3', metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(id='fe1235b0-8503-4856-8c1d-222ebf1b4a1c', metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.')]

In [24]:
smalldb.max_marginal_relevance_search(question, k=2, fetch_k=3)

[Document(id='adcaf1ee-3f76-4a78-b9ff-675e5a3514a3', metadata={}, page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(id='b3f73023-2e16-4b25-95c0-f37c219087ec', metadata={}, page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [28]:
question = "What is the email of Petr Olivka"

In [29]:
docs_ss = vectordb.similarity_search(question, k=3)

In [32]:
docs_ss[0].page_content[:200]

'Petr Olivka – IT Manager | Product Owner | SW Developer Residency:  Frýdek-Místek Nationality:   Česká republika Contacts:  +420 608 000 593, petr.olivka@gmail.com Professional profile Professional Ov'

In [33]:
docs_ss[1].page_content[:200]

'Petr Olivka – IT Manager | Product Owner | SW Developer Residency:  Frýdek-Místek Nationality:   Česká republika Contacts:  +420 608 000 593, petr.olivka@gmail.com Professional profile Professional Ov'

In [34]:
question = "What is the email of Petr Olivka?"

In [35]:
docs_mmr = vectordb.max_marginal_relevance_search(
    question,
    k=3,
)

In [36]:
for d in docs_mmr:
    print(d.metadata)

{'author': 'Petr Olivka', 'creationdate': "D:20250120142858Z00'00'", 'creator': 'Word', 'moddate': "D:20250120142858Z00'00'", 'page': 0, 'page_label': '1', 'producer': 'macOS Version 15.3 (Build 24D5040f) Quartz PDFContext', 'source': 'documents/CV_petr_olivka_general.pdf', 'title': 'CV_petr_olivka_general', 'total_pages': 5}
{'author': 'Petr Olivka', 'creationdate': "D:20250120142858Z00'00'", 'creator': 'Word', 'moddate': "D:20250120142858Z00'00'", 'page': 3, 'page_label': '4', 'producer': 'macOS Version 15.3 (Build 24D5040f) Quartz PDFContext', 'source': 'documents/CV_petr_olivka_general.pdf', 'title': 'CV_petr_olivka_general', 'total_pages': 5}
{'author': 'Petr Olivka', 'creationdate': "D:20250120142858Z00'00'", 'creator': 'Word', 'moddate': "D:20250120142858Z00'00'", 'page': 1, 'page_label': '2', 'producer': 'macOS Version 15.3 (Build 24D5040f) Quartz PDFContext', 'source': 'documents/CV_petr_olivka_general.pdf', 'title': 'CV_petr_olivka_general', 'total_pages': 5}


In [37]:
from langchain_openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [38]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The document 'documents/CV_petr_olivka_general.pdf' is a professional resume of Petr Olivka",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the resume",
        type="integer",
    ),
]

In [44]:
document_content_description = "Petr Olivka's CV"
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
retriever = SelfQueryRetriever().from_llm(
    llm=llm,
    vectorstore=vectordb,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    verbose=True
)

KeyError: 'vectorstore'